# Exercise 3 - Data Lake on S3

-> Needs an EMR cluster to work with S3

In [4]:
from pyspark.sql import SparkSession
import os
import configparser

# Make sure that your AWS credentials are loaded as env vars

In [5]:
config = configparser.ConfigParser()

#Normally this file should be in ~/.aws/credentials
config.read_file(open('aws/credentials.cfg'))

os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']

# Create spark session with hadoop-aws package

In [6]:
spark = SparkSession.builder\
                     .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                     .getOrCreate()

# Load data from S3

In [7]:
df = spark.read.csv("s3a://udacity-dend/pagila/payment/payment.csv")

In [8]:
df.printSchema()
df.show(5)

root
 |-- _c0: string (nullable = true)

+--------------------+
|                 _c0|
+--------------------+
|payment_id;custom...|
|16050;269;2;7;1.9...|
|16051;269;1;98;0....|
|16052;269;2;678;6...|
|16053;269;2;703;0...|
+--------------------+
only showing top 5 rows



# Infer schema, fix header and separator

In [9]:
df = spark.read.csv("s3a://udacity-dend/pagila/payment/payment.csv",sep=";", inferSchema=True, header=True)

In [10]:
df.printSchema()
df.show(5)

root
 |-- payment_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- staff_id: integer (nullable = true)
 |-- rental_id: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- payment_date: string (nullable = true)

+----------+-----------+--------+---------+------+--------------------+
|payment_id|customer_id|staff_id|rental_id|amount|        payment_date|
+----------+-----------+--------+---------+------+--------------------+
|     16050|        269|       2|        7|  1.99|2017-01-24 21:40:...|
|     16051|        269|       1|       98|  0.99|2017-01-25 15:16:...|
|     16052|        269|       2|      678|  6.99|2017-01-28 21:44:...|
|     16053|        269|       2|      703|  0.99|2017-01-29 00:58:...|
|     16054|        269|       1|      750|  4.99|2017-01-29 08:10:...|
+----------+-----------+--------+---------+------+--------------------+
only showing top 5 rows



# Fix the data yourself 

In [11]:
import  pyspark.sql.functions as F
dfPayment = df.withColumn('payment_timestamp', F.to_timestamp('payment_date'))
dfPayment.show(5)

+----------+-----------+--------+---------+------+--------------------+--------------------+
|payment_id|customer_id|staff_id|rental_id|amount|        payment_date|   payment_timestamp|
+----------+-----------+--------+---------+------+--------------------+--------------------+
|     16050|        269|       2|        7|  1.99|2017-01-24 21:40:...|2017-01-24 21:40:...|
|     16051|        269|       1|       98|  0.99|2017-01-25 15:16:...|2017-01-25 15:16:...|
|     16052|        269|       2|      678|  6.99|2017-01-28 21:44:...|2017-01-28 21:44:...|
|     16053|        269|       2|      703|  0.99|2017-01-29 00:58:...|2017-01-29 00:58:...|
|     16054|        269|       1|      750|  4.99|2017-01-29 08:10:...|2017-01-29 08:10:...|
+----------+-----------+--------+---------+------+--------------------+--------------------+
only showing top 5 rows



# Extract the month

In [13]:
dfPayment = dfPayment.withColumn('month', F.month('payment_timestamp'))
dfPayment.limit(10).toPandas()

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,payment_timestamp,month
0,16050,269,2,7,1.99,2017-01-24 21:40:19.996577+00,2017-01-24 21:40:19.996577,1
1,16051,269,1,98,0.99,2017-01-25 15:16:50.996577+00,2017-01-25 15:16:50.996577,1
2,16052,269,2,678,6.99,2017-01-28 21:44:14.996577+00,2017-01-28 21:44:14.996577,1
3,16053,269,2,703,0.99,2017-01-29 00:58:02.996577+00,2017-01-29 00:58:02.996577,1
4,16054,269,1,750,4.99,2017-01-29 08:10:06.996577+00,2017-01-29 08:10:06.996577,1
5,16055,269,2,1099,2.99,2017-01-31 12:23:14.996577+00,2017-01-31 12:23:14.996577,1
6,16056,270,1,193,1.99,2017-01-26 05:10:14.996577+00,2017-01-26 05:10:14.996577,1
7,16057,270,1,1040,4.99,2017-01-31 04:03:42.996577+00,2017-01-31 04:03:42.996577,1
8,16058,271,1,1096,8.99,2017-01-31 11:59:15.996577+00,2017-01-31 11:59:15.996577,1
9,16059,272,1,33,0.99,2017-01-25 02:47:17.996577+00,2017-01-25 02:47:17.996577,1


# Computer aggregate revenue per month

In [17]:
dfPayment.createOrReplaceTempView("payment")
spark.sql('''
    SELECT month,
           sum(amount) AS revenue
    FROM payment
    GROUP BY 1
    ORDER BY 2 DESC    
''').show()

+-----+------------------+
|month|           revenue|
+-----+------------------+
|    4|28559.460000003943|
|    3|23886.560000002115|
|    2| 9631.879999999608|
|    1| 4824.429999999856|
|    5|  514.180000000001|
+-----+------------------+



# Fix the schema

In [19]:
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl, StringType as Str, IntegerType as Int, DateType as Date
paymentSchema = R([
    Fld('payment_id', Int()),
    Fld('customer_id', Int()),
    Fld('staff_id', Int()),
    Fld('rental_id', Int()),
    Fld('amount', Dbl()),
    Fld('payment_date', Date()),
    ]
)

In [49]:
dfPaymentWithSchema = spark.read.csv('s3a://udacity-dend/pagila/payment/payment.csv',
                                     schema=paymentSchema, sep=';')

In [50]:
dfPaymentWithSchema.printSchema()
df.show(5)

root
 |-- payment_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- staff_id: integer (nullable = true)
 |-- rental_id: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- payment_date: date (nullable = true)

+----------+-----------+--------+---------+------+--------------------+
|payment_id|customer_id|staff_id|rental_id|amount|        payment_date|
+----------+-----------+--------+---------+------+--------------------+
|     16050|        269|       2|        7|  1.99|2017-01-24 21:40:...|
|     16051|        269|       1|       98|  0.99|2017-01-25 15:16:...|
|     16052|        269|       2|      678|  6.99|2017-01-28 21:44:...|
|     16053|        269|       2|      703|  0.99|2017-01-29 00:58:...|
|     16054|        269|       1|      750|  4.99|2017-01-29 08:10:...|
+----------+-----------+--------+---------+------+--------------------+
only showing top 5 rows



In [51]:
dfPaymentWithSchemaNotNull = dfPaymentWithSchema.na.drop(subset=['payment_id'])

In [54]:
dfPaymentWithSchema.createOrReplaceTempView("payment")
spark.sql('''
SELECT DATE_TRUNC('month', payment_date),
       SUM (amount)
FROM (
    SELECT * FROM payment
    WHERE amount IS NOT NULL) sub
GROUP BY 1
ORDER BY 2 DESC
''').show()

+--------------------------------------------------+------------------+
|date_trunc(month, CAST(payment_date AS TIMESTAMP))|       sum(amount)|
+--------------------------------------------------+------------------+
|                               2017-04-01 00:00:00|28559.460000003943|
|                               2017-03-01 00:00:00|23886.560000002115|
|                               2017-02-01 00:00:00| 9631.879999999608|
|                               2017-01-01 00:00:00| 4824.429999999856|
|                               2017-05-01 00:00:00|  514.180000000001|
+--------------------------------------------------+------------------+

